# Comparison of Similarity Metrics

This appendix compares the effect of all similarity metric implementations of the textdistance library [[TeDi](./A_References.ipynb#tedi)] for some example strings pair combinations for each feature to be calculated. The comparison is based on the goldstandard data and is the basis for deciding the similarity metrics for each feature to be used.

In [1]:
import textdistance as tedi

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Functions for Similarty Metrics Analysis](#Functions-for-Similarty-Metrics-Analysis)
- [Similarity Metric Assessments](#Similarity-Metric-Assessments)
    - [corporate](#corporate)
    - [exactDate](#exactDate)

## Data Takeover

As a first step, the training data set as a result of chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb) is read. Some sample strings of this data set will be used for the comparison and assessment of the different metrics implementations.

In [2]:
import os
import pandas as pd
import pickle as pk

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore results so far
df_feature_base = pd.read_pickle(os.path.join(path_goldstandard, 'feature_base_df.pkl'),
                                 compression=None)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
0,1,,,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],[],[],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula","grawechristian, graweursula",jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,reclam jun.,reclam jun.,,,"emma, roman","emma, roman",,,600 s.,600 s.
1,1,,,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],[],[],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,reclam jun.,reclam,,,"emma, roman",emma,,,600 s.,600 s.
2,1,,,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],[],[],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen,reclam jun.,reclam,,,"emma, roman","emma, roman",,,600 s.,600 s.
3,1,,,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],[],[],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,"grawechristian, graweursula",jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,reclam,reclam jun.,,,emma,"emma, roman",,,600 s.,600 s.
4,1,,,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],[],[],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,reclam,reclam,,,emma,emma,,,600 s.,600 s.


## Functions for Similarty Metrics Analysis

The available metrics algorithms of library [[TeDi](./A_References.ipynb#tedi)] are listed in the dictionary below. The dictionary will help calculating a similarity value for each implemented algorithm of the library.

In [3]:
tedi_algorithms = {
    # Edit based
    'Hamming' : tedi.Hamming(), 'MLIPNS' : tedi.MLIPNS(), 'Levenshtein' : tedi.Levenshtein(),
    'DamerauLevenshtein' : tedi.DamerauLevenshtein(), 'JaroWinkler' : tedi.JaroWinkler(),
    'StrCmp95' : tedi.StrCmp95(), 'NeedlemanWunsch' : tedi.NeedlemanWunsch(), 'Gotoh' : tedi.Gotoh(),
    'SmithWaterman' : tedi.SmithWaterman(),
    # Token based
    'Jaccard' : tedi.Jaccard(), 'Sorensen' : tedi.Sorensen(), 'Tversky' : tedi.Tversky(), 'Overlap' : tedi.Overlap(),
    'Tanimoto' : tedi.Tanimoto(), 'Cosine' : tedi.Cosine(), 'MongeElkan' : tedi.MongeElkan(), 'Bag' : tedi.Bag(),
    # Sequence based
    'LCSSeq' : tedi.LCSSeq(), 'LCSStr' : tedi.LCSStr(), 'RatcliffObershelp' : tedi.RatcliffObershelp(),
    # Compression based
    'ArithNCD' : tedi.ArithNCD(), 'RLENCD' : tedi.RLENCD(), 'BWTRLENCD' : tedi.BWTRLENCD(),
    'SqrtNCD' : tedi.SqrtNCD(), 'EntropyNCD' : tedi.EntropyNCD(), 'BZ2NCD' : tedi.BZ2NCD(),
    'LZMANCD' : tedi.LZMANCD(), 'ZLIBNCD' : tedi.ZLIBNCD(),
    # Phonetic
    'MRA' : tedi.MRA(), 'Editex' : tedi.Editex(),
    # Simple
    'Prefix' : tedi.Prefix(), 'Postfix' : tedi.Postfix(), 'Length' : tedi.Length(), 'Identity' : tedi.Identity(),
    'Matrix' : tedi.Matrix()
}

This appendix uses function $\texttt{.apply}\_\texttt{similarities()}$ that applies the $\texttt{.normalized}\_\texttt{similarity()}$ function of $\texttt{textdistance}$ for each algorithm object available in the library. The function is implemented in the separate code file [data_analysis_funcs.py](./data_analysis_funcs.py) 

In [4]:
import data_analysis_funcs as daf

## Similarity Metric Assessments

This section iterates through all available similarity metrics of library $\texttt{textdistance}$ and calculates the similarity values for a pair of two sample strings of each feature of the model. The calculated similarity values will be analysed visually and an algorithm will be decided to be used in chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb). The decision will be based on a visual assessment that is validated with the literature, [[Chri2012](./A_References.ipynb#chri2012)].

In [5]:
# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(tedi_algorithms)+3

number_of_samples = 30

### corporate

In [6]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'corporate_710_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD', 'Gotoh'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=number_of_samples)

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
34,allgemeine geschichtforschende gesellschaft de...,eidgenössisches topographisches bureau,0.054545,0.0,0.272727,0.290909,0.614052,0.636300,0.481818,0.131579,0.476190,0.645161,0.476190,0.789474,-1.070389,0.656218,0.007603,0.545455,0.345455,0.072727,0.279570,0.0,0.057692,0.412931,0.962395,0.500000,0.591837,0.277778,0.166667,0.409091,0.0,0.000000,0.690909,0.0,0.0
16,allgemeine geschichtforschende gesellschaft de...,société suisse pour la recherche en éducation,0.018182,0.0,0.163636,0.254545,0.626263,0.646465,0.481818,0.111111,0.492537,0.660000,0.492537,0.733333,-1.021695,0.663325,0.007769,0.600000,0.290909,0.054545,0.280000,0.0,0.057692,0.431842,0.956090,0.406250,0.510204,0.370370,0.000000,0.345455,0.0,0.000000,0.818182,0.0,0.0
7,allgemeine geschichtforschende gesellschaft de...,schweizerische gesellschaft für bildungsforsch...,0.010000,0.0,0.330000,0.330000,0.673979,0.679616,0.440000,0.163636,0.534653,0.696774,0.534653,0.981818,-0.903324,0.728136,0.009091,0.540000,0.340000,0.150000,0.425806,0.0,0.179775,0.441263,0.963925,0.494505,0.555556,0.439024,0.000000,0.440000,0.0,0.000000,0.550000,0.0,0.0
22,allgemeine geschichtforschende gesellschaft de...,eidgenössische landestopographie,0.036364,0.0,0.254545,0.309091,0.639935,0.649821,0.418182,0.125000,0.474576,0.643678,0.474576,0.875000,-1.075288,0.667424,0.008264,0.509091,0.327273,0.072727,0.275862,0.0,0.038462,0.420424,0.964676,0.453125,0.510204,0.296296,0.166667,0.400000,0.0,0.000000,0.581818,0.0,0.0
11,allgemeine geschichtforschende gesellschaft de...,"staatsoper (wien), wiener philharmoniker, chor",0.018182,0.0,0.181818,0.218182,0.544474,0.568426,0.509091,0.108696,0.422535,0.594059,0.422535,0.652174,-1.242857,0.596432,0.007769,0.545455,0.327273,0.054545,0.158416,0.0,0.000000,0.440355,0.939105,0.437500,0.551020,0.370370,0.000000,0.309091,0.0,0.000000,0.836364,0.0,0.0
33,allgemeine geschichtforschende gesellschaft de...,"schweiz, eidgenössisches topographisches bureau",0.054545,0.0,0.236364,0.236364,0.605760,0.629435,0.545455,0.148936,0.545455,0.705882,0.545455,0.765957,-0.874469,0.708064,0.007934,0.654545,0.363636,0.127273,0.137255,0.0,0.057692,0.470999,0.954645,0.492537,0.632653,0.333333,0.000000,0.354545,0.0,0.000000,0.854545,0.0,0.0
3,allgemeine geschichtforschende gesellschaft de...,"metropolitan opera (new york), metropolitan op...",0.050633,0.0,0.189873,0.202532,0.526877,0.545381,0.443038,0.127273,0.313725,0.477612,0.313725,0.581818,-1.672425,0.485462,0.007769,0.405063,0.265823,0.037975,0.238806,0.0,0.039474,0.369452,0.918092,0.402778,0.510204,0.368421,0.166667,0.297468,0.0,0.000000,0.696203,0.0,0.0
12,allgemeine geschichtforschende gesellschaft de...,wiener philharmoniker,0.018182,0.0,0.181818,0.181818,0.559271,0.572431,0.281818,0.047619,0.310345,0.473684,0.310345,0.857143,-1.688056,0.529641,0.005785,0.327273,0.200000,0.054545,0.210526,0.0,-0.019231,0.302279,0.901759,0.406250,0.510204,0.259259,0.000000,0.345455,0.0,0.000000,0.381818,0.0,0.0
29,allgemeine geschichtforschende gesellschaft de...,allgemeine geschichtforschende gesellschaft de...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.307692,0.585786,1.000000,0.890625,0.959184,0.944444,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0
31,allgemeine geschichtforschende gesellschaft de...,schweiz,0.018182,0.0,0.127273,0.127273,0.468398,0.484502,0.127273,1.000000,0.127273,0.225806,0.127273,1.000000,-2.974005,0.356753,0.004959,0.127273,0.127273,0.127273,0.225806,0.0,0.076923,0.171329,0.781835,0.437500,0.510204,0.222222,0.000000,0.245455,0.0,0.127273,0.127273,0.0,0.0


### exactDate

In [10]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'exactDate_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=number_of_samples)

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
18,1950uuuu,19911990,0.250,0.0,0.250,0.250,0.500000,0.500000,0.6250,0.6250,0.250,0.230769,0.375,0.230769,0.375,-2.115477,0.375,0.023438,0.375,0.375,0.250,0.375,0.000000,0.333333,0.346962,0.752081,0.766667,0.840000,0.500000,0.400000,0.625,0.250,0.000,1.0,0.0,0.0
48,1950uuuu,1981uuuu,0.750,1.0,0.750,0.750,0.866667,0.833333,0.8750,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000000,0.375000,0.406587,0.890727,0.866667,0.840000,0.500000,0.500000,0.750,0.250,0.500,1.0,0.0,0.0
85,1950uuuu,2014uuuu,0.500,0.0,0.500,0.500,0.777778,0.777778,0.7500,0.7500,0.500,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.625,0.500,0.625,0.000000,0.375000,0.390524,0.916667,0.833333,0.840000,0.500000,0.000000,0.500,0.000,0.500,1.0,0.0,0.0
27,1950uuuu,1843uuuu,0.625,1.0,0.625,0.625,0.775000,0.750000,0.8125,0.8125,0.625,0.454545,0.625,0.454545,0.625,-1.137504,0.625,0.039062,0.625,0.625,0.500,0.625,0.000000,0.375000,0.292893,0.875000,0.800000,0.840000,0.500000,0.250000,0.625,0.125,0.500,1.0,0.0,0.0
82,1950uuuu,20111201,0.000,0.0,0.000,0.000,0.333333,0.333333,0.5000,0.5000,0.000,0.142857,0.250,0.142857,0.250,-2.807355,0.250,0.015625,0.250,0.250,0.125,0.250,0.000000,0.000000,0.241016,0.715928,0.633333,0.840000,0.500000,0.000000,0.375,0.000,0.000,1.0,0.0,0.0
59,1950uuuu,1941uuuu,0.750,1.0,0.750,0.750,0.866667,0.833333,0.8750,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.046875,0.750,0.750,0.500,0.750,0.000000,0.375000,0.406587,0.890727,0.866667,0.840000,0.500000,0.500000,0.750,0.250,0.500,1.0,0.0,0.0
86,1950uuuu,201310uu,0.250,0.0,0.250,0.250,0.583333,0.583333,0.6250,0.6250,0.250,0.333333,0.500,0.333333,0.500,-1.584963,0.500,0.046875,0.500,0.500,0.375,0.500,0.000000,0.111111,0.373087,0.865761,0.766667,0.840000,0.500000,0.000000,0.500,0.000,0.250,1.0,0.0,0.0
78,1950uuuu,19829999,0.250,0.0,0.250,0.250,0.500000,0.500000,0.6250,0.6250,0.250,0.142857,0.250,0.142857,0.250,-2.807355,0.250,0.015625,0.250,0.250,0.250,0.250,0.000000,0.222222,0.228727,0.714385,0.766667,0.840000,0.416667,0.400000,0.625,0.250,0.000,1.0,0.0,0.0
42,1950uuuu,1990uuuu,0.875,1.0,0.875,0.875,0.933333,0.916667,0.9375,0.9375,0.875,0.777778,0.875,0.777778,0.875,-0.362570,0.875,0.054688,0.875,0.875,0.625,0.875,0.000000,0.375000,0.504218,0.932393,0.900000,0.760000,0.583333,0.500000,0.875,0.250,0.625,1.0,0.0,0.0
46,1950uuuu,20091990,0.000,0.0,0.000,0.125,0.333333,0.333333,0.5000,0.5000,0.000,0.230769,0.375,0.230769,0.375,-2.115477,0.375,0.023438,0.375,0.375,0.250,0.375,0.000000,0.111111,0.341648,0.812093,0.700000,0.840000,0.428571,0.000000,0.375,0.000,0.000,1.0,0.0,0.0


Attribute $\texttt{exactDate}$ is a string of four digits or characters. For calculating the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance), each string pair is compared character-wise. A so called edit distance between the string pair is calculated as the sum of all edit operations needed to convert the strings into each other, [[Chri2012](./A_References.ipynb#chri2012)]. The resulting Hamming similarity can be deduced from the edit distance and the length of one string. This can be easily done in the examples of the DataFrame above. The Hamming similarity shall be used for attribute $\texttt{exactDate}$.

The Hamming similarity has one drawbak, though, looking at the goldstandard data. The attribute may be filled with letter 'u' for 'unknown' digits instead of a number. A letter 'u' will result in an edit distance of 1. This is a statement which need not be true for the bibliographic units that the records describe. On the other hand, a pair of strings with a letter 'u' at the same digit, need not have a distance of 0 for the bibliographical units. For this reason, the Hamming similarity will be adapted for the case of existence of letter 'u' in one of the strings of the pair. The Hamming similarity will be increased by a value of 1/16 for each unknown digit in a string and the maximum of unknown digits of the compared pair will be calculated. This algorithm based on the Hamming similarity will be implemented in chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb).